In [30]:
import mne
import mne_bids
import os, os.path as op

In [23]:
t1_bids_path=mne_bids.get_bids_path_from_fname('/data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/anat/sub-ON02811_ses-01_acq-MPRAGE_T1w.nii.gz')

In [74]:
bids_root = '/data/MEG_workshop_data/ds004215'
deriv_root = op.join(bids_root, 'derivatives')
fs_subjects_dir = op.join(deriv_root, 'freesurfer','subjects')
# fs_subjects_dir = op.join(bids_root, 'derivatives', 'freesurfer','subjects')

In [77]:
bids_deriv_path.root

PosixPath('/data/MEG_workshop_data/ds004215/derivatives/MEGworkshop')

In [78]:
subject = 'ON02811'
bids_path = mne_bids.BIDSPath(root=bids_root, subject = subject, session='01', task='airpuff', run='01', datatype='meg')
bids_deriv_path = bids_path.copy().update(root=op.join(bids_root, 'derivatives', 'MEGworkshop'), check=False)
if not op.exists(bids_deriv_path.root): os.mkdir(bids_deriv_path.root)

#t1_bids_path = bids_path.copy().update(datatype='anat', session=None, task=None, run='01')

In [26]:
t1_bids_path.fpath

PosixPath('/data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/anat/sub-ON02811_ses-01_acq-MPRAGE_T1w.nii.gz')

In [6]:
raw = mne_bids.read_raw_bids(bids_path)

ds directory : /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -1.82   70.43    0.00 mm <->   -1.82   70.43    0.00 mm (orig :  -45.24   54.14 -275.77 mm) diff =    0.000 mm
       1.82  -70.43    0.00 mm <->    1.82  -70.43    0.00 mm (orig :   62.56  -36.39 -269.47 mm) diff =    0.000 mm
      90.00    0.00    0.00 mm <->   90.00   -0.00   -0.00 mm (orig :   65.74   74.78 -250.30 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds/sub-ON02811_ses-01_task-airpuff_run-01_meg.meg4: 
    System clock channel is available, checking which

/tmp/ipykernel_6689/3316973082.py:1: RuntimeWarning: The unit for channel(s) SCLK01-177, UADC001-2104, UPPT001, trigger has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


In [7]:
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True)

ds directory : /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -1.82   70.43    0.00 mm <->   -1.82   70.43    0.00 mm (orig :  -45.24   54.14 -275.77 mm) diff =    0.000 mm
       1.82  -70.43    0.00 mm <->    1.82  -70.43    0.00 mm (orig :   62.56  -36.39 -269.47 mm) diff =    0.000 mm
      90.00    0.00    0.00 mm <->   90.00   -0.00   -0.00 mm (orig :   65.74   74.78 -250.30 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds/sub-ON02811_ses-01_task-airpuff_run-01_meg.meg4: 
    System clock channel is available, checking which

PosixPath('/data/MEG_workshop_data/ds004215/sub-ON028747/ses-01/meg/sub-ON028747_ses-01_task-airpuff_run-01')

In [29]:
raw.ch_names

['SCLK01',
 'BG1',
 'BG2',
 'BG3',
 'BP1',
 'BP2',
 'BP3',
 'BR1',
 'BR2',
 'BR3',
 'G11',
 'G12',
 'G13',
 'G22',
 'G23',
 'P11',
 'P12',
 'P22',
 'P23',
 'Q11',
 'Q12',
 'Q13',
 'Q22',
 'Q23',
 'R11',
 'R12',
 'R13',
 'R22',
 'R23',
 'MLC11',
 'MLC12',
 'MLC13',
 'MLC14',
 'MLC15',
 'MLC16',
 'MLC17',
 'MLC21',
 'MLC22',
 'MLC23',
 'MLC24',
 'MLC25',
 'MLC31',
 'MLC32',
 'MLC41',
 'MLC42',
 'MLC51',
 'MLC52',
 'MLC53',
 'MLC54',
 'MLC55',
 'MLC61',
 'MLC62',
 'MLC63',
 'MLF11',
 'MLF12',
 'MLF13',
 'MLF14',
 'MLF21',
 'MLF22',
 'MLF23',
 'MLF24',
 'MLF31',
 'MLF32',
 'MLF33',
 'MLF34',
 'MLF35',
 'MLF41',
 'MLF42',
 'MLF43',
 'MLF44',
 'MLF45',
 'MLF46',
 'MLF51',
 'MLF52',
 'MLF53',
 'MLF54',
 'MLF55',
 'MLF56',
 'MLF61',
 'MLF62',
 'MLF63',
 'MLF64',
 'MLF65',
 'MLF66',
 'MLF67',
 'MLO11',
 'MLO12',
 'MLO13',
 'MLO14',
 'MLO21',
 'MLO22',
 'MLO23',
 'MLO24',
 'MLO31',
 'MLO32',
 'MLO33',
 'MLO34',
 'MLO41',
 'MLO42',
 'MLO43',
 'MLO44',
 'MLO51',
 'MLO52',
 'MLO53',
 'MLP11',
 'MLP

In [41]:
trans = mne_bids.get_head_mri_trans(bids_path, extra_params={'clean_names': True}, t1_bids_path=t1_bids_path, fs_subject='sub-'+subject, fs_subjects_dir=fs_subjects_dir)

ds directory : /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -1.82   70.43    0.00 mm <->   -1.82   70.43    0.00 mm (orig :  -45.24   54.14 -275.77 mm) diff =    0.000 mm
       1.82  -70.43    0.00 mm <->    1.82  -70.43    0.00 mm (orig :   62.56  -36.39 -269.47 mm) diff =    0.000 mm
      90.00    0.00    0.00 mm <->   90.00   -0.00   -0.00 mm (orig :   65.74   74.78 -250.30 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /data/MEG_workshop_data/ds004215/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_meg.ds/sub-ON02811_ses-01_task-airpuff_run-01_meg.meg4: 
    System clock channel is available, checking which

/tmp/ipykernel_6689/2944433602.py:1: RuntimeWarning: The unit for channel(s) SCLK01-177, UADC001-2104, UPPT001, trigger has changed from V to NA.
  trans = mne_bids.get_head_mri_trans(bids_path, extra_params={'clean_names': True}, t1_bids_path=t1_bids_path, fs_subject='sub-'+subject, fs_subjects_dir=fs_subjects_dir)


In [42]:
trans

<Transform | head->MRI (surface RAS)>
[[ 0.99795308  0.03128865 -0.0557733   0.00288806]
 [-0.05788118  0.81276317 -0.57971182  0.00432162]
 [ 0.02719208  0.58175343  0.81291054 -0.03190548]
 [ 0.          0.          0.          1.        ]]

### Prepare Imaging for Source Localization

In [50]:
# fwd_name=f'{output_dir}/{subjid}-fwd.fif'
fwd_name = bids_deriv_path.copy().update(extension='.fif', suffix='fwd')

In [ ]:
#Requires Freesurfer -  This is already done for our subject
#mne.bem.make_watershed_bem(subjid, subjects_dir=subjects_dir)

## Do a use case for calc_mnetrans

In [ ]:
import nih2mne
from nih2mne import calc_mnetrans

In [ ]:
trans_fname = f'{topdir}/{subjid}-trans.fif'
if not os.path.exists(trans_fname):
    # The calc_mnetrans.py code takes tagged Afni, Brainsight, or BIDS json
    anat_json = f'{topdir}/{subjid}/ses-01/meg/{subjid}_ses-01_task-airpuff_run-01_coordsystem.json'
    ! calc_mnetrans.py -anat_json {anat_json} -trans_output {subjid}-trans.fif -dsname {meg_fname} -subject {subjid} -subjects_dir {subjects_dir}
    trans = mne.read_trans(trans_fname)
else:
    trans = mne.read_trans(trans_fname)

### Plot the transform

In [60]:
mne.viz.plot_alignment(raw.info, trans=trans, subject='sub-'+subject, subjects_dir=fs_subjects_dir, dig=True)

Using outer_skin.surf for head surface.
Getting helmet for system CTF_275
Removing 5 compensators from info because not all compensation channels were picked.
Channel types::	mag: 272


In [67]:
# fwd_name=f'{output_dir}/{subjid}_vol-fwd.fif'
vol_fwd_name = fwd_name = bids_deriv_path.copy().update(extension='.fif', suffix='volfwd')
if not os.path.exists(fwd_name):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    bem = mne.make_bem_model('sub-'+subject, subjects_dir=fs_subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    src = mne.source_space.setup_volume_source_space(subject='sub-'+subject, subjects_dir=fs_subjects_dir,
                                                     mri='T1.mgz', bem=bem_sol) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(fwd_name, forward)
else:
    forward = mne.read_forward_solution(fwd_name)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
inner skull CM is  -2.60 -26.24  17.24 mm
Surfaces passed the basic topology checks.
Complete.

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (2562) -> inner skull (2562) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.
BEM              : <ConductorModel | BEM (1 layer) solver=mne>
grid                  : 5.0 mm
mindist               : 5.0 mm
MRI volume            : /data/MEG_workshop_data/ds004215/derivatives/freesurfer/subjects/sub-ON02811/mri/T1.mgz

Reading /data/MEG_workshop_data/ds004215/derivatives/freesurfer/subjects/sub-ON02811/mri/T1.mgz...

Taking inner skull from <ConductorModel | BEM (1 layer) solver=mne>
Surface CM = (  -2.6  -26.2   17.2) mm
Surface fits inside a sphere with radius   92.1 mm
Surface extent:
    x =  -75.2 ...   69.8 mm
    y = 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   12.9s remaining:   25.9s


    Found    0/7270 points outside using solid angles
    Total 9596/9596 points inside the surface
Interior check completed in 13387.1 ms

Checking surface interior status for 300 points...
    Found   0/300 points inside  an interior sphere of radius   41.1 mm
    Found 300/300 points outside an exterior sphere of radius   92.1 mm
    Found   0/  0 points outside using surface Qhull


[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   13.2s remaining:    9.4s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:   13.3s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   13.4s finished


    Found   0/  0 points outside using solid angles
    Total 0/300 points inside the surface
Interior check completed in 69.7 ms

Composing the field computation matrix...


...

Computing MEG at 9596 source locations (free orientations)...


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.8s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    3.9s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    4.1s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    4.3s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.4s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.



Finished.


[Parallel(n_jobs=12)]: Done   4 out of  12 | elapsed:    3.4s remaining:    6.8s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:    3.4s remaining:    2.4s
[Parallel(n_jobs=12)]: Done  10 out of  12 | elapsed:    3.4s remaining:    0.7s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    3.5s finished
/tmp/ipykernel_6689/3304805438.py:11: RuntimeWarning: This filename (/data/MEG_workshop_data/ds004215/derivatives/MEGworkshop/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_volfwd.fif) does not conform to MNE naming conventions. All forward files should end with -fwd.fif, -fwd.fif.gz, _fwd.fif or _fwd.fif.gz
  mne.forward.write_forward_solution(fwd_name, forward)


FileNotFoundError: [Errno 2] No such file or directory: '/data/MEG_workshop_data/ds004215/derivatives/MEGworkshop/sub-ON02811/ses-01/meg/sub-ON02811_ses-01_task-airpuff_run-01_volfwd.fif'

## Volumetric Source Model 

In [ ]:
src = forward['src']
src.plot(subjid, subjects_dir=subjects_dir, brain=False, trans=trans)

### Surface Source

In [ ]:
fwd_name=f'{output_dir}/{subjid}-fwd.fif'
if not os.path.exists(fwd_name):
    # Make a 1 layer bem from the inner skull - Appropriate for MEG, bad for EEG
    bem = mne.make_bem_model(subjid, subjects_dir=subjects_dir, conductivity=[0.3])
    bem_sol = mne.make_bem_solution(bem)
    src = mne.source_space.setup_source_space(subject=subjid, subjects_dir=subjects_dir, n_jobs=-1) 
    forward = mne.make_forward_solution(raw.info, trans, src, 
                                        bem_sol, meg=True, eeg=False, n_jobs=-1)
    mne.forward.write_forward_solution(fwd_name, forward)
else:
    forward = mne.read_forward_solution(fwd_name)

In [ ]:
src = forward['src']
src.plot(subjects_dir=subjects_dir, head=False, brain='white', trans=trans)

## Boundary Element Model (For MEG - red boundary in figure is all that is necessary)

In [ ]:
%matplotlib inline
_ = mne.viz.plot_bem(subject=subjid, subjects_dir=subjects_dir)

## Spatial relationship of the brain to the MEG sensors

In [ ]:
mne.viz.plot_alignment(raw.info, trans=trans, subject=subjid, subjects_dir=subjects_dir, dig=True, surfaces='pial')

## What is a forward Model?
## Simulation of all the source locations through the BEM

In [ ]:
print(forward['sol']['data'].shape)
print('Channels, Source Locations X 3')

In [ ]:
from mne.viz.topomap import plot_topomap

In [ ]:
tmp=raw.copy()
grads = [i for i in tmp.ch_names if i[0]=='M']
_=tmp.pick_channels(grads)

In [ ]:
%matplotlib qt
fig, axs = plt.subplots(3,2)
axs[0,0].set_title('Location 1 Forward Model')
plot_topomap(forward['sol']['data'][:,1003], tmp.info, axes=axs[0,0])
plot_topomap(forward['sol']['data'][:,1004], tmp.info, axes=axs[1,0])
plot_topomap(forward['sol']['data'][:,1005], tmp.info, axes=axs[2,0])

axs[0,1].set_title('Location 2 Forward Model')
plot_topomap(forward['sol']['data'][:,5003], tmp.info, axes=axs[0,1])
plot_topomap(forward['sol']['data'][:,5004], tmp.info, axes=axs[1,1])
plot_topomap(forward['sol']['data'][:,5005], tmp.info, axes=axs[2,1])


In [ ]:
%matplotlib inline
fig=pylab.figure(figsize=(12,8))
pylab.imshow(forward['sol']['data'], aspect='auto', cmap='jet', vmax=3e-6, vmin=-3e-6)
pylab.title('Forward Model Simulation at every point in the source model')
pylab.xlabel('Source Location')
pylab.ylabel('Channel index')